# Introduction to Data Science, CS 5963 / Math 3900
*CS 5963 / MATH 3900, University of Utah, http://datasciencecourse.net/* 

# Lab 18: Relational Databases

Up to now, we've mainly used flat tables to store and process data. Most structured data in the real world, however, is stored in databases, and specifically in [relational databases](https://en.wikipedia.org/wiki/Relational_database). Other database types have specific advantages, such as performance (NoSQL), suitable for graphs (graph databases, e.g., Neo4j), or compatibility with in-memory OO data structures (Object Oriented Databases). Relational databases (and their implementations - Relational Database Management Systems (RDBMS) - are dominant. 

What's the purpose of a relational database? It separates data into multiple tables to avoid redundancy. Here is a simple example for an online transaction. Here's the flat table that describes the data:

| Product Name | Price | Product Description | Customer Name | Customer Address| 
| ------------- | - | - | - | -| 
| MacBook | 2700 | MacBook Pro 15' | Jack Black | San Francisco |  
| Dongle HDMI | 40 | USB-C to HDMI | Jack Black | San Francisco | 
| Dongle USB-B | 40 | USB-C to USB-B | Jack Black | San Francisco | 

The problem here is that we store the name of the customer multiple times, and, if multiple customers purchase the same item, also all the information about the products. Obviously, that's neither great from a storage efficiency perspective nor from an update perspective. If Jack Black wants to update his address, for example, than we'd have to update all three rows. 

A different approach is to separate this table into multiple, meaningful tables. We'll introduce a Customers, Products and a Transactions table. And we'll introduce keys for each row in this table:


Table Customers:

| CID | Customer Name | Customer Address | 
| - | - | - |
| 1 | Jack Black | San Francisco |

Table Products:

| PID | Product Name | Price | Product Description |
| - | - | - | - | 
| 1 | MacBook | 2700 | MacBook Pro 15' |
| 2 | Dongle HDMI | 40 | USB-C to HDMI |
| 3 | Dongle USB-B | 40 | USB-C to USB-B |

CID and PID are called **primary keys**, as they uniquely identify the row in the table. 

The Transaction Table now refers to the products and the customers only by their keys, which are called **foreign keys** in this context, as they are the primary keys of a "foreign" table.

Table Transactions: 

| TID | PID | CID | 
| - | - | - |
| 1 | 1 | 1 | 
| 2 | 3 | 1 | 
| 3 | 2 | 1 | 

Now, if Jack Black wants to update his address, or if the seller wants to update the product description, we can do this in a single place. 

Of course, we have to do a little more work to regenerate (an interesting subset) of transactions, as we had it before. In this lab we'll learn how to do that and more. 

## RDBMS Transactions

RDBMS are designed to support **read, write, update** transactions, in addition to operations that **create tables**, etc. Database transactions. In practice, for a database to remain consistent, these transactions have to guarantee [certain properties](https://en.wikipedia.org/wiki/ACID). We'll largely ignore updates, creating tables, etc. - these are operations that you'll likely need if you build a web-application. For the purpose of our data science class, we'll stick to reading/querying for data, as we want to learn **how to get data out of a database**. You can learn more about databases in [CS 6530](https://www.cs.utah.edu/~lifeifei/cs6530/) (or the undergrad version CS 5530).


## SQL - The Structured Query Language


[SQL](https://en.wikipedia.org/wiki/SQL) is a domain specific language to execute the transactions described above. We'll look mainly at queries, to read and aggregate data. 

Here is a very simple query:

```SQL
SELECT * FROM products
```

This statement selects and retreives all rows (`*`) from the table `products`.

We can restrict the lines to retrieve with a `WHERE` clause:

```SQL
SELECT * FROM products WHERE 'Price' > 100
```

This retrieves all the rows where the price is higher than 100. 

We'll go through more specific SQL statements on real examples. You can learn more about SQL [online](https://www.codecademy.com/learn/learn-sql).


## SQLite

Most database management systems are implemented as client-server systems, i.e., the database is hosted on a dedicated server where multiple users can read and write. Examples are [PostgreSQL](https://www.postgresql.org/), [MySQL](https://www.mysql.com/) or [Oracle](https://www.oracle.com/database/index.html). These database management systems are fast and can handle multiple users withouth causing conflicts. However, they require a separate server and installation, so we'll use [SQLite](https://sqlite.org/) an alternative that works for single users and stores the whole database in a single file.  SQLite is widely used in desktop installations and will serve our purposes. 

We'll also use the [sqlite python interface](https://docs.python.org/3/library/sqlite3.html).

## Sample Database

This tutorial uses the SQLite sample database found [here](http://www.sqlitetutorial.net/sqlite-sample-database/).

Here is a chart of the database schema: 

![](database_schema.png)

 * `employees` table stores employees data such as employee id, last name, first name, etc. It also has a field named `ReportsTo` to specify who reports to whom.
 * `customers` table stores customers data.
 * `invoices & invoice_items` tables: these two tables store invoice data. The invoices table stores invoice header data and the invoice_items table stores the invoice line items data.
 * `artists` table stores artists data. It is a simple table that contains only artist id and name.
 * `albums` table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
 * `media_types` table stores media types such as MPEG audio file, ACC audio file, etc.
 * `genres` table stores music types such as rock, jazz, metal, etc.
 * `tracks` table store the data of songs. Each track belongs to one album.
 * `playlists` & `playlist_track` tables: playlists table store data about playlists. Each playlist contains a list of tracks. Each track may belong to multiple playlists. The relationship between the playlists table and tracks table is many-to-many. The playlist_track table is used to reflect this relationship.
 
The diagram above highlights the **primary keys** in each table and the relationships, using **foreign keys** to each other table. Note that `employees` has a self-reference, to capture the reports-to relationship.

## Querying the Table

In [1]:
import pandas as pd
import sqlite3 as sq

# we connect to the database, which - in the case of sqlite - is a local file
conn = sq.connect("./chinook.db")
# we retreive the "cursor" on the database.
c = conn.cursor()

# now we can execute a SQL statement
c.execute("SELECT * FROM albums")
# and print the first line from the result
print(c.fetchone())

(1, 'For Those About To Rock We Salute You', 1)


Here, we used a cursor for accessing the data. To retrieve data after executing a SELECT statement, you can either treat the cursor as an iterator, call the cursor’s `fetchone()` method to retrieve a single matching row, or call `fetchall()` to get a list of the matching rows.

This is how you use it as an iterator: 

In [2]:
for row in c:
    print(row)

(2, 'Balls to the Wall', 2)
(3, 'Restless and Wild', 2)
(4, 'Let There Be Rock', 1)
(5, 'Big Ones', 3)
(6, 'Jagged Little Pill', 4)
(7, 'Facelift', 5)
(8, 'Warner 25 Anos', 6)
(9, 'Plays Metallica By Four Cellos', 7)
(10, 'Audioslave', 8)
(11, 'Out Of Exile', 8)
(12, 'BackBeat Soundtrack', 9)
(13, 'The Best Of Billy Cobham', 10)
(14, 'Alcohol Fueled Brewtality Live! [Disc 1]', 11)
(15, 'Alcohol Fueled Brewtality Live! [Disc 2]', 11)
(16, 'Black Sabbath', 12)
(17, 'Black Sabbath Vol. 4 (Remaster)', 12)
(18, 'Body Count', 13)
(19, 'Chemical Wedding', 14)
(20, 'The Best Of Buddy Guy - The Millenium Collection', 15)
(21, 'Prenda Minha', 16)
(22, 'Sozinho Remix Ao Vivo', 16)
(23, 'Minha Historia', 17)
(24, 'Afrociberdelia', 18)
(25, 'Da Lama Ao Caos', 18)
(26, 'Acústico MTV [Live]', 19)
(27, 'Cidade Negra - Hits', 19)
(28, 'Na Pista', 20)
(29, 'Axé Bahia 2001', 21)
(30, 'BBC Sessions [Disc 1] [Live]', 22)
(31, 'Bongo Fury', 23)
(32, 'Carnaval 2001', 21)
(33, 'Chill: Brazil (Disc 1)', 24)
(3

And here we retrieve all the data at once:

In [3]:
c.execute("SELECT * FROM albums")
print(c.fetchall())

[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3), (6, 'Jagged Little Pill', 4), (7, 'Facelift', 5), (8, 'Warner 25 Anos', 6), (9, 'Plays Metallica By Four Cellos', 7), (10, 'Audioslave', 8), (11, 'Out Of Exile', 8), (12, 'BackBeat Soundtrack', 9), (13, 'The Best Of Billy Cobham', 10), (14, 'Alcohol Fueled Brewtality Live! [Disc 1]', 11), (15, 'Alcohol Fueled Brewtality Live! [Disc 2]', 11), (16, 'Black Sabbath', 12), (17, 'Black Sabbath Vol. 4 (Remaster)', 12), (18, 'Body Count', 13), (19, 'Chemical Wedding', 14), (20, 'The Best Of Buddy Guy - The Millenium Collection', 15), (21, 'Prenda Minha', 16), (22, 'Sozinho Remix Ao Vivo', 16), (23, 'Minha Historia', 17), (24, 'Afrociberdelia', 18), (25, 'Da Lama Ao Caos', 18), (26, 'Acústico MTV [Live]', 19), (27, 'Cidade Negra - Hits', 19), (28, 'Na Pista', 20), (29, 'Axé Bahia 2001', 21), (30, 'BBC Sessions [Disc 1] [Live]', 22), (31, 'B

An alternative approach that fits well into our previous workflows is to use pandas to store the table directly, using the [`read_sql()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html) function:

In [4]:
# this is similar to the read_csv()function
pd.read_sql("""SELECT * FROM albums""", conn).head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


When we **specify individual columns** instead of using the `*`, we get only those columns:

In [5]:
pd.read_sql("""SELECT Title FROM albums""", conn).head()

,Title
0,For Those About To Rock We Salute You
1,Balls to the Wall
2,Restless and Wild
3,Let There Be Rock
4,Big Ones


We can run more **restrictive queries** with the `WHERE` keyword:

In [6]:
# note that the triple quotes allow us to break lines
pd.read_sql("""SELECT * FROM albums 
                    WHERE Title = 'Big Ones'""", conn)

,AlbumId,Title,ArtistId
0,5,Big Ones,3


## Security

We can use python variables to use in the columns, however, you typically shouldn't trust the content of the variables, especially in a user facing system. Imagine, you read in which attribute to query for from a website field where a user can specify a name. The user could use this to attack your SQL server using [SQL Injection](https://en.wikipedia.org/wiki/SQL_injection). For example, for this statement:

In [7]:
title = "Big Ones"
# executescript runs multiple sql statements. 
# This isn't very helpful for quering, but can be great for other operations.
c.executescript("""SELECT * FROM albums WHERE Title = '""" + title + "'")

Let's look at the tables in the database:

In [8]:
pd.read_sql("""SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
            """, conn)

,name
0,albums
1,artists
2,customers
3,employees
4,genres
5,invoices
6,media_types
7,playlist_track
8,playlists
9,sqlite_sequence


Now, if we were to read `title` from a user, and adversary user could write: 

```SQL
a';DROP TABLE invoice_items;
```

In [9]:
title = "a';DROP TABLE invoice_items;"
print("""SELECT * FROM albums WHERE Title = '""" + title + "'")
c.executescript("""SELECT * FROM albums WHERE Title = '""" + title + "'")

SELECT * FROM albums WHERE Title = 'a';DROP TABLE invoice_items;'


OperationalError: no such table: invoice_items

This throws an error, but the table is gone:

In [10]:
pd.read_sql("""SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
            """, conn)

,name
0,albums
1,artists
2,customers
3,employees
4,genres
5,invoices
6,media_types
7,playlist_track
8,playlists
9,sqlite_sequence


Instead, you should use a ? for substitution, and pass in a tuple. This will make sure that no additional statements will be executed. 

In [11]:
title = ("a';DROP TABLE invoice_items;",)
# this will result in an error
c.executescript('SELECT * FROM albums WHERE title=?', title)

TypeError: function takes exactly 1 argument (2 given)

Here is a working example with safe code:

In [12]:
title = ("Big Ones",)
# this will result in an error
c.execute('SELECT * FROM albums WHERE title=?', title)
c.fetchone()

(5, 'Big Ones', 3)

Of course that's not all to say about making your code secure, but it's a start! 

![](exploits_of_a_mom.png)

## Aggregating Queries

There are many ways to aggregate and group by with SQL, and we won't be able to cover them in detail. We'll use a simple example for an `INNER JOIN` to resolve a foreign key relationship here.

We want to query tracks from the tracks table and add the titles of the tracks into the resulting table.  Let's take a looks at the tracks table:

In [ ]:
pd.read_sql("""SELECT * FROM tracks;""", conn).head(5)

This is the relationship we care about:

![](albums_tracks_tables.jpg)

An INNER JOIN simply merges tables based on a primary key / foreign key relationship. 

Here is an example:

In [13]:
pd.read_sql("""SELECT
 trackid,
 name as "Track Name",
 title as "Album Title"
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid;""", conn).head(20)

,TrackId,Track Name,Album Title
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,6,Put The Finger On You,For Those About To Rock We Salute You
2,7,Let's Get It Up,For Those About To Rock We Salute You
3,8,Inject The Venom,For Those About To Rock We Salute You
4,9,Snowballed,For Those About To Rock We Salute You
5,10,Evil Walks,For Those About To Rock We Salute You
6,11,C.O.D.,For Those About To Rock We Salute You
7,12,Breaking The Rules,For Those About To Rock We Salute You
8,13,Night Of The Long Knives,For Those About To Rock We Salute You
9,14,Spellbound,For Those About To Rock We Salute You


Let's take this statement apart:

```SQL
 SELECT
 trackid,
 name as "Track Name",
 title as "Album Title"
```
Here, we chose what to select, and we also rename the columns for presentation purposes with the `as` statement. Note that we use column names from both tables, the tracks and the album table! 

```SQL
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid;
```

This is where the magic happens: we say that the tracks and albums table should be joined on the `albums.albumid = tracks.albumid` relationship.



Using INNER JOIN, you can construct flat tables out of databases that you then can use in the data science process. SQL is of course much more powerful, you can do mathematical operations, group-by/aggregates, etc. However, we can do most of this on a dataframe also, but doing a join like that would be rather painful without the INNER JOIN SQL statement.